# Fusion entre la base de données pré-existantes reviews.csv ainsi que la nouvelle base de donnée issue de www.beerwulf.com

In [1]:
# Importation des modules nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
df_bw = pd.read_csv('data_beerwulf')
df_bw.describe()

,Unnamed: 0,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte
count,1130.000000,1130.000000,1130.000000,645.000000,1068.000000,1068.000000,1068.000000,1068.000000,960.000000,1051.000000,960.000000
mean,564.500000,33.269912,6.350088,35.848062,2.976592,2.624532,0.884831,2.251873,2.882219,3.491227,4.332208
std,326.347208,3.920001,2.074320,20.030920,0.811362,0.821705,1.081587,1.008514,0.692156,0.269224,0.931041
min,0.000000,25.000000,0.200000,4.000000,0.000000,0.000000,0.000000,0.000000,1.190000,2.000000,1.900000
25%,282.250000,33.000000,5.000000,22.000000,3.000000,2.000000,0.000000,2.000000,2.490000,3.350000,3.780000
50%,564.500000,33.000000,6.100000,32.000000,3.000000,3.000000,1.250000,2.000000,2.790000,3.510000,4.230000
75%,846.750000,33.000000,7.500000,45.000000,3.000000,3.000000,1.250000,3.000000,3.090000,3.660000,4.580000
max,1129.000000,65.000000,16.500000,220.000000,5.000000,5.000000,6.250000,5.000000,7.990000,4.450000,11.800000
